<a href="https://colab.research.google.com/github/22ananya/MUSI6201/blob/main/Final%20Project%20-%20Stereo%20Demixing/Run_Eval_on_Saved_Mixes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import generally required packages - update as needed

In [ ]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as sp
import scipy.io.wavfile as wav

Import/Install the prerequisite code for implementing the Cadenza challenge - includes baselines, other important files

In [ ]:
!pip install pyclarity==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 55.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 44.2 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filena

Import the dataset for the Cadenza challenge directly through the Google Drive link - Only needs to be done once! So now commented out


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check current path

In [ ]:
import os
print(os.getcwd())

/content


Change current path to Cadenza folder

In [ ]:
os.chdir('/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation')
print(os.getcwd())

/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation


#Process single audio file through the entire model step by step

Import all dependencies - same as enhance.py file

In [ ]:
from __future__ import annotations

import json
import logging
from pathlib import Path

# pylint: disable=import-error
import hydra
import numpy as np
import torch
from numpy import ndarray
from omegaconf import DictConfig
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB

from clarity.enhancer.compressor import Compressor
from clarity.enhancer.nalr import NALR
from clarity.evaluator.haaqi import compute_haaqi
from clarity.utils.audiogram import Audiogram, Listener
from clarity.utils.file_io import read_signal
from clarity.utils.flac_encoder import FlacEncoder
from clarity.utils.results_support import ResultsFile
from clarity.utils.signal_processing import (
    clip_signal,
    denormalize_signals,
    normalize_signal,
    resample,
    to_16bit,
    compute_rms,
    resample,
)
from clarity.utils.source_separation_support import get_device, separate_sources
from recipes.cad_icassp_2024.baseline.evaluate import (
    apply_gains,
    apply_ha,
    make_scene_listener_list,
    remix_stems,
    load_reference_stems,
)

logger = logging.getLogger(__name__)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/evaluate.py:190: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import all the required functions defined in Enhance.py that do not need to be changed

In [ ]:
from recipes.cad_icassp_2024.baseline.enhance import (
    save_flac_signal,
    decompose_signal,
    process_remix_for_listener
)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/enhance.py:182: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import the correct config file (hardcoded location)

In [ ]:
from omegaconf import OmegaConf
config = OmegaConf.load('config.yaml')
print("default source sep model: ", config.separator.model)

default source sep model:  demucs


Set input directory

In [ ]:
reference_folder = Path("Val_Ref_Tracks")
enhanced_mix_folder = Path("Val_enhanced_signals_demucs_100")

# load the audio files in provided path

In [ ]:
enhanced_files = [] # initialize list of enhanced audio files
reference_files = [] # initialize list of reference files
for file in os.listdir(enhanced_mix_folder): # iterate over all files in the directory
    if file.endswith('.flac'): # if the file is an audio file
        enhanced_files.append(os.path.join(enhanced_mix_folder, file)) # add the file to the list of audio files
        reference_files.append(os.path.join(reference_folder, file.replace('.flac','_ref.flac'))) # add the corresponding ref file to the list of ref files

In [ ]:
print(enhanced_files[0])
print(reference_files[0])
print(len(enhanced_files))

Val_enhanced_signals_demucs_100/scene_50001_L5020_remix.flac
Val_Ref_Tracks/scene_50001_L5020_remix_ref.flac
100


Load listener data

In [ ]:
config.path.root = '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation'


In [ ]:
# Load listener audiograms and songs
listener_dict = Listener.load_listener_dict(config.path.listeners_file)

with Path(config.path.gains_file).open("r", encoding="utf-8") as file:
    gains = json.load(file)

with Path(config.path.scenes_file).open("r", encoding="utf-8") as file:
    scenes = json.load(file)

with Path(config.path.scene_listeners_file).open("r", encoding="utf-8") as file:
    scenes_listeners = json.load(file)

with Path(config.path.music_file).open("r", encoding="utf-8") as file:
    songs = json.load(file)

Create a list of songs, listeners (audiogram) and head position (hrtf) to generate, or evaluate the mix - based on provided data

In [ ]:
# Select a batch to process
scene_listener_pairs = make_scene_listener_list(
    scenes_listeners, config.evaluate.small_test
)

scene_listener_pairs = scene_listener_pairs[
    config.evaluate.batch :: config.evaluate.batch_size
]

In [ ]:
print(scene_listener_pairs[0])

('scene_50001', 'L5020')


# Load functions required for audio evaluation

Add functions and variables needed for evaluation and scoring

In [ ]:
scores_headers = [
    "scene",
    "song",
    "listener",
    "left_score",
    "right_score",
    "score",
]


results_file = ResultsFile(
            "Val_scores_HAAQI_demucs.csv",
            header_columns=scores_headers,
)

# Process Audio - Currently set to process a fixed number of runs (listener - scene pairings, can be changed to run entire dataset) - HAAQI is very very slow

In [ ]:
from recipes.cad_icassp_2024.baseline.evaluate import level_normalisation
previous_song = ""
num_scenes = len(scene_listener_pairs)
nrun = 1 # change to num_scenes for full dataset


for idx in range(nrun):
    # Extract track, listener, scene info from file name
    file_name = enhanced_files[idx]
    sname = file_name.split("/", 1)[1].split("_L",1)[0]
    lname = "L" +  file_name.split("/", 1)[1].split("_L",1)[1].split("_")[0]

    # Get the listener's audiogram
    listener = listener_dict[lname]
    scene = scenes[sname]
    song_name = f"{scene['music']}-{scene['head_loudspeaker_positions']}"

    print(sname)
    print(lname)

    # load audio files for HAAQI
    reference_mixture = read_signal(
            filename=reference_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    enhanced_mixture = read_signal(
            filename=enhanced_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    # renormalize files wrt each other in case they're not at the same level
    enhanced_mixture = level_normalisation(enhanced_mixture, reference_mixture, config.sample_rate)

    # Evaluate - compare with the generated mixes
    # Compute the scores
    left_score = compute_haaqi(
        processed_signal=resample(
            enhanced_mixture[:, 0],
            config.remix_sample_rate,
            config.HAAQI_sample_rate,
        ),
        reference_signal=resample(
            reference_mixture[:, 0], config.sample_rate, config.HAAQI_sample_rate
        ),
        processed_sample_rate=config.HAAQI_sample_rate,
        reference_sample_rate=config.HAAQI_sample_rate,
        audiogram=listener.audiogram_left,
        equalisation=2,
        level1=65 - 20 * np.log10(compute_rms(reference_mixture[:, 0])),
    )

    right_score = compute_haaqi(
        processed_signal=resample(
            enhanced_mixture[:, 1],
            config.remix_sample_rate,
            config.HAAQI_sample_rate,
        ),
        reference_signal=resample(
            reference_mixture[:, 1], config.sample_rate, config.HAAQI_sample_rate
        ),
        processed_sample_rate=config.HAAQI_sample_rate,
        reference_sample_rate=config.HAAQI_sample_rate,
        audiogram=listener.audiogram_right,
        equalisation=2,
        level1=65 - 20 * np.log10(compute_rms(reference_mixture[:, 1])),
    )

        # Save scores
    results_file.add_result(
        {
            "scene": sname,
            "song": song_name,
            "listener": listener.id,
            "left_score": left_score,
            "right_score": right_score,
            "score": float(np.mean([left_score, right_score])),
        }
    )

    print(right_score, left_score)


scene_50001
L5020
0.1461050385209223 0.12756825527721488
